*This notebook is from https://github.com/neubig/anlp-code* by Graham Neubig

We added additional printing of feature weights in the Error Analysis section.

# Training a Sentiment Classifier

This is a notebook for [LUH Advanced NLP](https://sites.google.com/view/jen-web/sose-2025) that trains a sentiment classifier based on data. Specifically, it uses a bag-of-words to extract features, and the structured perceptron algorithm to train the classifier.

It will take in a text `X` and return a `label` of "1" if the sentiment of the text is positive, "-1" if the sentiment of the text is negative, and "0" if the sentiment of the text is neutral. You can test the accuracy of your classifier on the [Stanford Sentiment Treebank](http://nlp.stanford.edu/sentiment/index.html) by running the notebook all the way to end.

## Setup

Setup code, do imports.

In [1]:
import random
import tqdm

## Feature Extraction

Feature extraction code, how do we get the features we use in training? By default we just use every word.

In [2]:
def extract_features(x: str) -> dict[str, float]:
    features = {}
    x_split = x.split(' ')
    for x in x_split:
        features[x] = features.get(x, 0) + 1.0
    return features

Also, initialize the feature weights to zero.

In [3]:

feature_weights = {}

## Data Reading

Read in the data from the training and dev (or finally test) sets

In [4]:
def read_xy_data(filename: str) -> tuple[list[str], list[int]]:
    x_data = []
    y_data = []
    with open(filename, 'r') as f:
        for line in f:
            label, text = line.strip().split(' ||| ')
            x_data.append(text)
            y_data.append(int(label))
    return x_data, y_data

In [5]:
x_train, y_train = read_xy_data('./data/train.txt')
x_dev, y_dev = read_xy_data('./data/dev.txt')

In [6]:
print(x_train[0])
print(y_train[0])

The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal .
1


## Inference Code

How we run the classifier.

In [7]:
def run_classifier(features: dict[str, float]) -> int:
    score = 0
    for feat_name, feat_value in features.items():
        score = score + feat_value * feature_weights.get(feat_name, 0)
    if score > 0:
        return 1
    elif score < 0:
        return -1
    else:
        return 0

## Training Code

Learn the weights of the classifier.

In [8]:
NUM_EPOCHS = 5
for epoch in range(1, NUM_EPOCHS+1):
    # Shuffle the order of the data
    data_ids = list(range(len(x_train)))
    random.shuffle(data_ids)
    # Run over all data points
    for data_id in tqdm.tqdm(data_ids, desc=f'Epoch {epoch}'):
        x = x_train[data_id]
        y = y_train[data_id]
        # We will skip neutral examples
        if y == 0:    
            continue
        # Make a prediction
        features = extract_features(x)
        predicted_y = run_classifier(features)
        # Update the weights if the prediction is wrong
        if predicted_y != y:
            for feature in features:
                feature_weights[feature] = feature_weights.get(feature, 0) + y * features[feature]

Epoch 5: 100%|██████████| 8544/8544 [00:00<00:00, 258920.37it/s]


## Evaluation Code

How we evaluate the classifier:

In [9]:
def calculate_accuracy(x_data: list[str], y_data: list[int]) -> float:
    total_number = 0
    correct_number = 0
    for x, y in zip(x_data, y_data):
        y_pred = run_classifier(extract_features(x))
        total_number += 1
        if y == y_pred:
            correct_number += 1
    return correct_number / float(total_number)

In [10]:
label_count = {}
for y in y_dev:
    if y not in label_count:
        label_count[y] = 0
    label_count[y] += 1
print(label_count)

{1: 444, 0: 229, -1: 428}


In [11]:
train_accuracy = calculate_accuracy(x_train, y_train)
test_accuracy = calculate_accuracy(x_dev, y_dev)
print(f'Train accuracy: {train_accuracy}')
print(f'Dev/test accuracy: {test_accuracy}')

Train accuracy: 0.766619850187266
Dev/test accuracy: 0.5821980018165305


In [ ]:
#sort the feature weights and print the top 10 and bottom 10

## Error Analysis

An important part of improving any system is figuring out where it goes wrong. The following two functions allow you to randomly observe some mistaken examples, which may help you improve the classifier. Feel free to write more sophisticated methods for error analysis as well.

In [12]:
def get_feature_contributions(features):
    output = {}
    for feat_name, feat_value in features.items():
        output[feat_name] = feat_value * feature_weights.get(feat_name, 0)
    return output

def find_errors(x_data, y_data):
    error_ids = []
    y_preds = []
    id2contributions = {}
    for i, (x, y) in enumerate(zip(x_data, y_data)):
        features = extract_features(x)
        y_preds.append(run_classifier(features))
        if y != y_preds[-1]:
            error_ids.append(i)
            id2contributions[i] = get_feature_contributions(features)
    for _ in range(5):
        my_id = random.choice(error_ids)
        x, y, y_pred = x_data[my_id], y_data[my_id], y_preds[my_id]

        print(f'{x}\ntrue label: {y}\npredicted label: {y_pred}')
        contributions = sorted(id2contributions[my_id].items(), key=lambda x: -x[1])
        for feat_name, contribution in contributions:
            print(f'Feature: {feat_name} ({contribution})')
        
        print()

In [13]:
find_errors(x_dev, y_dev)

Despite the evocative aesthetics evincing the hollow state of modern love life , the film never percolates beyond a monotonous whine .
true label: -1
predicted label: 1
Feature: love (8.0)
Feature: evocative (5.0)
Feature: the (3.0)
Feature: modern (3.0)
Feature: film (3.0)
Feature: aesthetics (1.0)
Feature: life (1.0)
Feature: evincing (0.0)
Feature: state (0.0)
Feature: never (0.0)
Feature: percolates (0.0)
Feature: a (0.0)
Feature: . (0.0)
Feature: Despite (-1.0)
Feature: of (-1.0)
Feature: , (-1.0)
Feature: whine (-1.0)
Feature: hollow (-4.0)
Feature: beyond (-4.0)
Feature: monotonous (-4.0)

The primitive force of this film seems to bubble up from the vast collective memory of the combatants .
true label: 1
predicted label: -1
Feature: force (6.0)
Feature: film (3.0)
Feature: The (2.0)
Feature: the (2.0)
Feature: from (1.0)
Feature: primitive (0.0)
Feature: this (0.0)
Feature: bubble (0.0)
Feature: combatants (0.0)
Feature: . (0.0)
Feature: up (-1.0)
Feature: collective (-1.0)
Fea

## Visualize feature weights

We can inspect the weights that were learned for various features. Below we show the largest, smallest, and randomly selected feature weights. Inspecting them may give insight into the learned classifier.

In [14]:
import random

k = 25
topk_features = sorted(feature_weights.items(), key=lambda x: -x[1])[:k]
bottomk_features = sorted(feature_weights.items(), key=lambda x: x[1])[:k]
randomk_features = random.sample(list(feature_weights.items()), k)

print("Top-k")
for feature in topk_features:
    print(feature)

print("\nBottom-k")
for feature in bottomk_features:
    print(feature)

print("\nRandom k")
for feature in randomk_features:
    print(feature)

Top-k
('solid', 13.0)
('remarkable', 13.0)
('appealing', 12.0)
('powerful', 12.0)
('rare', 11.0)
('sweet', 11.0)
('terrific', 11.0)
('portrait', 11.0)
('refreshing', 11.0)
('brilliant', 10.0)
('hilarious', 10.0)
('beautifully', 10.0)
('somewhat', 10.0)
('going', 9.0)
('human', 9.0)
('wonderful', 9.0)
('eyes', 9.0)
('fun', 9.0)
('works', 9.0)
('deeply', 9.0)
('perfectly', 9.0)
('wonderfully', 9.0)
('sharp', 9.0)
('entertaining', 9.0)
('follow', 9.0)

Bottom-k
('stupid', -14.0)
('were', -14.0)
('worst', -13.0)
('suffers', -13.0)
('TV', -12.0)
('repetitive', -12.0)
('mess', -12.0)
('left', -11.0)
('idea', -11.0)
('Lawrence', -11.0)
('pretentious', -11.0)
('lousy', -11.0)
('none', -11.0)
('contrived', -10.0)
('Unfortunately', -10.0)
('lacking', -10.0)
('flat', -10.0)
('dull', -10.0)
('Feels', -10.0)
('Sheridan', -10.0)
('instead', -10.0)
('violence', -10.0)
('scene', -10.0)
("n't", -9.0)
('bore', -9.0)

Random k
('documentarian', 1.0)
('Beavis', 1.0)
('happened', -3.0)
('ticket', 1.0)
('Ex